In [ ]:
import os
#setting up plaidml as backend to use my local gpu in Macbook pro
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend" 


In [ ]:
os.chdir('/Users/nasifimtiaz/Desktop/Caltech256-project')
print(os.getcwd())

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

#images are in different shapes. needs to square them.
def shrink_square(img, shrink_size, color_mode, fill_color):
    #all the images will be shrinked to shrink_size with any leftover filled with color_fill
    ## thumbnail resizes image while maintaining aspect ratio
    shrinked_img=img.thumbnail((shrink_size,shrink_size),Image.ANTIALIAS)
    
    #if some images were originally less than shrink_size, bring to shrink size by filling the leftover
    output=Image.new(color_mode,(shrink_size,shrink_size),fill_color)
    output.paste(img, (int((shrink_size - img.size[0]) / 2), int((shrink_size - img.size[1]) / 2)))
    
    return output


#turn images to np_array
def image_to_tensors(img):
    arr=np.array(img)
    return arr

        

In [ ]:
hm={}
path='./256_ObjectCategories'
folders=os.listdir(path)

for folder in folders:
    os.chdir(path+'/'+folder)
    image_paths=os.listdir()
    class_images=[]
    for i in image_paths:
        if i[-4:]=='.jpg':
            print("processing ",i)
            img=Image.open(i)
            shrinked_img=shrink_square(img, 128, 'RGB', 0)
            img_arr=image_to_tensors(shrinked_img)
            img_arr=img_arr/255 #scaling pixel values to [0,1]
            class_images.append(img_arr)
            img.close()
            shrinked_img.close()
    class_name= folder.split('.')[1]
    hm[class_name]=class_images
    os.chdir('../..')


In [ ]:
print("total classes: ", len(hm))
sum=0
flawed=0
class_freqs=[]
for k in hm.keys():
    sum+=len(hm[k])
    class_freqs.append(len(hm[k]))
    for i in hm[k]:
        if i.shape != (128,128,3):
            flawed+=1
print("total images: ",sum)
print("flawd images: ", flawed)

In [ ]:
#visualize frequency per class categories
classes=hm.keys()
y_pos=np.arange(len(classes))
plt.figure(figsize=(20,5))
plt.bar(y_pos,class_freqs,align='center', alpha=1)
plt.xticks(y_pos,classes,rotation=90)
plt.ylabel('# of images')
plt.title('Image number per classes. While few classes have higher images, most has around 100 images')
plt.show()




In [ ]:
#create a class_mapping_to_int
class_to_int_mapping={}
int_to_class_mapping={}
count=0
for k in hm.keys():
    count+=1
    class_to_int_mapping[k]=count
    int_to_class_mapping[count]=k

In [ ]:
#convert hm to X,y data mapping
#where each row of X is an image, y is it's class in int
import random
def make_Xy(hm,subset_classes):
    rand=random.sample(range(1,len(hm.keys())),subset_classes)
    X=[]
    y=[]
    for r in rand:
        images=hm[int_to_class_mapping[r]]
        flatten_images=[]
        for i in images:
            flatten_images.append(i.flatten())
        X.extend(flatten_images)
        y.extend([r]*len(flatten_images))
    X=np.matrix(X)
    y=np.array(y)
    
    return X,y        